<img src="images/banner.png" style="width: 100%;">

# Working with Databases I Notebook 2

References:

[1] McKinney, Wes. *Python for data analysis.* " O'Reilly Media, Inc.", 2022.

[2] Teate, Renee MP. *SQL for Data Scientists: A Beginner's Guide for Building Datasets for Analysis.* John Wiley & Sons, 2021.

[3] Forta, Ben. *Sams Teach Yourself SQL in 10 Minutes a Day, 5th Edition*. O'Reilly Media, Inc., 2020

[4] Python sqlite3 documentation - https://docs.python.org/3/library/sqlite3.html

[5] Revised and grammar checked using ChatGPT - https://chatgpt.com/

Prepared by: Leodegario Lorenzo II

From the previous notebook, we've learned how to select the data you want from a database and filter the rows that you want. In most cases, the data that you want will not reside in just 1 table, but you may need to join multiple tables, then possibly aggregate and group common values from certain columns to perform your desired analysis.

In this next notebook, we'll cover how we join tables, perform data aggregation, and performing group by operations.

In the previous notebook, we learned how to retrieve data from a database using `SELECT` statements then filtering rows using conditions combined with the `WHERE` clause. In practice, however, the data you need rarely exist in a single table. More often, meaningful analysis requires combining data from multiple tables then summarizing it through group-by or aggregation operations.

In this notebook, we will build on those foundations by learning how to join SQL tables, aggregate data, and group records using the `GROUP BY` clause.

In [1]:
import pandas as pd
import sqlalchemy as sqla

We'll continue to use the farmer's market database as our database for examples.

In [2]:
db = sqla.create_engine('sqlite:///data/farmers_market.db')

## 1 SQL `JOIN`s

You may have noticed that some tables reference foreign key `id`s from other tables. This design helps optimize storage, since storing integers is more efficient than repeatedly storing strings. However, this can make analysis more difficult for downstream users when only numeric `id`s are visible.

For this reason, it is often desirable to join multiple tables so that these ids can be replaced with their corresponding, human-readable values.

Let's say we want to list each product name along with its product category name. For this scenario, we might want to join the `product` table with the `product_category` table.

In [3]:
query = """
        SELECT *
        FROM product
        """
product = pd.read_sql(query, db)
product

,product_id,product_name,product_size,product_category_id,product_qty_type
0,1,Habanero Peppers - Organic,medium,1,lbs
1,2,Jalapeno Peppers - Organic,small,1,lbs
2,3,Poblano Peppers - Organic,large,1,unit
3,4,Banana Peppers - Jar,8 oz,3,unit
4,5,Whole Wheat Bread,1.5 lbs,3,unit
5,6,Cut Zinnias Bouquet,medium,5,unit
6,7,Apple Pie,"10""",3,unit
7,8,Cherry Pie,"10""",3,unit
8,9,Sweet Potatoes,medium,1,lbs
9,10,Eggs,1 dozen,6,unit


In [22]:
query = """
        SELECT *
        FROM product_category
        """
product_category = pd.read_sql(query, db)
product_category

,product_category_id,product_category_name
0,1,Fresh Fruits & Vegetables
1,2,Packaged Pantry Goods
2,3,Packaged Prepared Food
3,4,Freshly Prepared Food
4,5,Plants & Flowers
5,6,Eggs & Meat (Fresh or Frozen)
6,7,Non-Edible Products


To join them we must figure out what type of `JOIN` we want to use then setting a conditoin for tables to join `ON`. There are four main types of SQL joins:

<img src="images/sql_joins.png" style="width: 60%;">

The `LEFT JOIN` pulls all records from the "left side" of the `JOIN` then matches records based from the table on the "right side" of the join based on the provided criteria.

Let's try to demonstrate this in action by using `LEFT JOIN` on the `product` and `product_category` table. The syntax for joining SQL tables is:

```
SELECT <columns to return>
FROM <left table>
<JOIN TYPE> <right table>
ON <left table>.<field in left table to match> = <right table>.<field in right table to match>
```

In [10]:
query = """
        SELECT
            *
        FROM product
            LEFT JOIN product_category
                ON product.product_category_id = product_category.product_category_id
        """
pd.read_sql(query, db)

,product_id,product_name,product_size,product_category_id,product_qty_type,product_category_id,product_category_name
0,1,Habanero Peppers - Organic,medium,1,lbs,1,Fresh Fruits & Vegetables
1,2,Jalapeno Peppers - Organic,small,1,lbs,1,Fresh Fruits & Vegetables
2,3,Poblano Peppers - Organic,large,1,unit,1,Fresh Fruits & Vegetables
3,4,Banana Peppers - Jar,8 oz,3,unit,3,Packaged Prepared Food
4,5,Whole Wheat Bread,1.5 lbs,3,unit,3,Packaged Prepared Food
5,6,Cut Zinnias Bouquet,medium,5,unit,5,Plants & Flowers
6,7,Apple Pie,"10""",3,unit,3,Packaged Prepared Food
7,8,Cherry Pie,"10""",3,unit,3,Packaged Prepared Food
8,9,Sweet Potatoes,medium,1,lbs,1,Fresh Fruits & Vegetables
9,10,Eggs,1 dozen,6,unit,6,Eggs & Meat (Fresh or Frozen)


All of the rows of the left table has been return and matched according to the criteria specified on the `ON` condition with the rows of the right table.

We further refine our query and result by doing the following:

1. Specifying columns to be returned.
2. Specifying aliases to the left and right tables (for easy referencing)
3. Sepcifying aliases to repeated columns.

In [52]:
query = """
        SELECT
            p.product_id,
            p.product_name,
            p.product_size,
            p.product_category_id,
            p.product_qty_type,
            pc.product_category_name
        FROM product AS p
            LEFT JOIN product_category AS pc
                ON p.product_category_id = pc.product_category_id
        """
pd.read_sql(query, db)

,product_id,product_name,product_size,product_category_id,product_qty_type,product_category_name
0,1,Habanero Peppers - Organic,medium,1,lbs,Fresh Fruits & Vegetables
1,2,Jalapeno Peppers - Organic,small,1,lbs,Fresh Fruits & Vegetables
2,3,Poblano Peppers - Organic,large,1,unit,Fresh Fruits & Vegetables
3,4,Banana Peppers - Jar,8 oz,3,unit,Packaged Prepared Food
4,5,Whole Wheat Bread,1.5 lbs,3,unit,Packaged Prepared Food
5,6,Cut Zinnias Bouquet,medium,5,unit,Plants & Flowers
6,7,Apple Pie,"10""",3,unit,Packaged Prepared Food
7,8,Cherry Pie,"10""",3,unit,Packaged Prepared Food
8,9,Sweet Potatoes,medium,1,lbs,Fresh Fruits & Vegetables
9,10,Eggs,1 dozen,6,unit,Eggs & Meat (Fresh or Frozen)


In [47]:
pd.merge(product, product_category, on='product_category_id', how='left')

,product_id,product_name,product_size,product_category_id,product_qty_type,product_category_name
0,1,Habanero Peppers - Organic,medium,1,lbs,Fresh Fruits & Vegetables
1,2,Jalapeno Peppers - Organic,small,1,lbs,Fresh Fruits & Vegetables
2,3,Poblano Peppers - Organic,large,1,unit,Fresh Fruits & Vegetables
3,4,Banana Peppers - Jar,8 oz,3,unit,Packaged Prepared Food
4,5,Whole Wheat Bread,1.5 lbs,3,unit,Packaged Prepared Food
5,6,Cut Zinnias Bouquet,medium,5,unit,Plants & Flowers
6,7,Apple Pie,"10""",3,unit,Packaged Prepared Food
7,8,Cherry Pie,"10""",3,unit,Packaged Prepared Food
8,9,Sweet Potatoes,medium,1,lbs,Fresh Fruits & Vegetables
9,10,Eggs,1 dozen,6,unit,Eggs & Meat (Fresh or Frozen)


Next, we demonstrate the `RIGHT JOIN`, which returns all the rows from the "right table", along with matching rows from the "left table, using the field specified by the `ON` part of the query.

In [16]:
query = """
        SELECT
            p.product_id,
            p.product_name,
            pc.product_category_id,
            pc.product_category_name
        FROM product AS p
            RIGHT JOIN product_category AS pc
                ON p.product_category_id = pc.product_category_id
        """
pd.read_sql(query, db)

,product_id,product_name,product_category_id,product_category_name
0,1.0,Habanero Peppers - Organic,1,Fresh Fruits & Vegetables
1,2.0,Jalapeno Peppers - Organic,1,Fresh Fruits & Vegetables
2,3.0,Poblano Peppers - Organic,1,Fresh Fruits & Vegetables
3,4.0,Banana Peppers - Jar,3,Packaged Prepared Food
4,5.0,Whole Wheat Bread,3,Packaged Prepared Food
5,6.0,Cut Zinnias Bouquet,5,Plants & Flowers
6,7.0,Apple Pie,3,Packaged Prepared Food
7,8.0,Cherry Pie,3,Packaged Prepared Food
8,9.0,Sweet Potatoes,1,Fresh Fruits & Vegetables
9,10.0,Eggs,6,Eggs & Meat (Fresh or Frozen)


In [40]:
pd.merge(product, product_category, on='product_category_id', how='right')

,product_id,product_name,product_size,product_category_id,product_qty_type,product_category_name
0,1.0,Habanero Peppers - Organic,medium,1,lbs,Fresh Fruits & Vegetables
1,2.0,Jalapeno Peppers - Organic,small,1,lbs,Fresh Fruits & Vegetables
2,3.0,Poblano Peppers - Organic,large,1,unit,Fresh Fruits & Vegetables
3,9.0,Sweet Potatoes,medium,1,lbs,Fresh Fruits & Vegetables
4,12.0,Baby Salad Lettuce Mix - Bag,1/2 lb,1,unit,Fresh Fruits & Vegetables
5,13.0,Baby Salad Lettuce Mix,1 lb,1,lbs,Fresh Fruits & Vegetables
6,14.0,Red Potatoes,None,1,None,Fresh Fruits & Vegetables
7,15.0,Red Potatoes - Small,,1,None,Fresh Fruits & Vegetables
8,16.0,Sweet Corn,Ear,1,unit,Fresh Fruits & Vegetables
9,17.0,Carrots,sold by weight,1,lbs,Fresh Fruits & Vegetables


In [17]:
query = """
        SELECT
            p.product_id,
            p.product_name,
            pc.product_category_id,
            pc.product_category_name
        FROM product AS p
            INNER JOIN product_category AS pc
                ON p.product_category_id = pc.product_category_id
        """
pd.read_sql(query, db)

,product_id,product_name,product_category_id,product_category_name
0,1,Habanero Peppers - Organic,1,Fresh Fruits & Vegetables
1,2,Jalapeno Peppers - Organic,1,Fresh Fruits & Vegetables
2,3,Poblano Peppers - Organic,1,Fresh Fruits & Vegetables
3,4,Banana Peppers - Jar,3,Packaged Prepared Food
4,5,Whole Wheat Bread,3,Packaged Prepared Food
5,6,Cut Zinnias Bouquet,5,Plants & Flowers
6,7,Apple Pie,3,Packaged Prepared Food
7,8,Cherry Pie,3,Packaged Prepared Food
8,9,Sweet Potatoes,1,Fresh Fruits & Vegetables
9,10,Eggs,6,Eggs & Meat (Fresh or Frozen)


In [42]:
pd.merge(product, product_category, on='product_category_id', how='inner')

,product_id,product_name,product_size,product_category_id,product_qty_type,product_category_name
0,1,Habanero Peppers - Organic,medium,1,lbs,Fresh Fruits & Vegetables
1,2,Jalapeno Peppers - Organic,small,1,lbs,Fresh Fruits & Vegetables
2,3,Poblano Peppers - Organic,large,1,unit,Fresh Fruits & Vegetables
3,4,Banana Peppers - Jar,8 oz,3,unit,Packaged Prepared Food
4,5,Whole Wheat Bread,1.5 lbs,3,unit,Packaged Prepared Food
5,6,Cut Zinnias Bouquet,medium,5,unit,Plants & Flowers
6,7,Apple Pie,"10""",3,unit,Packaged Prepared Food
7,8,Cherry Pie,"10""",3,unit,Packaged Prepared Food
8,9,Sweet Potatoes,medium,1,lbs,Fresh Fruits & Vegetables
9,10,Eggs,1 dozen,6,unit,Eggs & Meat (Fresh or Frozen)


### `JOIN`s with More than Two Tables

We can also join multiple tables if by chaining multiple SQL `JOIN` claues.

For example, say, we want to show all of the `booth`s in the farmer's market along with the assigned vendor including booths that aren't assigned to a vendor yet.

In [56]:
query = """
        SELECT
            b.booth_number,
            b.booth_type,
            vba.market_date,
            v.vendor_id,
            v.vendor_name,
            v.vendor_type
        FROM booth AS b
            LEFT JOIN vendor_booth_assignments AS vba
                ON b.booth_number = vba.booth_number
            LEFT JOIN vendor AS v
                ON v.vendor_id = vba.vendor_id
        ORDER BY b.booth_number, vba.market_date
        """
pd.read_sql(query, db)

,booth_number,booth_type,market_date,vendor_id,vendor_name,vendor_type
0,1,Standard,2019-04-03,3.0,Mountain View Vegetables,Fresh Variety: Veggies & More
1,1,Standard,2019-04-06,3.0,Mountain View Vegetables,Fresh Variety: Veggies & More
2,1,Standard,2019-04-10,3.0,Mountain View Vegetables,Fresh Variety: Veggies & More
3,1,Standard,2019-04-13,3.0,Mountain View Vegetables,Fresh Variety: Veggies & More
4,1,Standard,2019-04-17,3.0,Mountain View Vegetables,Fresh Variety: Veggies & More
...,...,...,...,...,...,...
921,11,Large,2020-09-30,7.0,Marco`s Peppers,Fresh Focused
922,11,Large,2020-10-03,7.0,Marco`s Peppers,Fresh Focused
923,11,Large,2020-10-07,7.0,Marco`s Peppers,Fresh Focused
924,11,Large,2020-10-10,7.0,Marco`s Peppers,Fresh Focused


Looks like some of the booth's haven't been assigned to any vendor yet. Let's make a query to investigate this further.

In [57]:
query = """
        SELECT
            b.booth_number,
            b.booth_type,
            vba.market_date,
            v.vendor_id,
            v.vendor_name,
            v.vendor_type
        FROM booth AS b
            LEFT JOIN vendor_booth_assignments AS vba
                ON b.booth_number = vba.booth_number
            LEFT JOIN vendor as v
                ON v.vendor_id = vba.vendor_id
        WHERE v.vendor_id IS NULL
        ORDER BY b.booth_number, vba.market_date
        """
pd.read_sql(query, db)

,booth_number,booth_type,market_date,vendor_id,vendor_name,vendor_type
0,3,Small,None,None,None,None
1,4,Small,None,None,None,None
2,5,Small,None,None,None,None
3,9,Small,None,None,None,None
4,12,Large,None,None,None,None


We can interact with databases using Python in several ways. For this demonstration, we will show two ways, one using `sqlite3` and another using `pandas` through `sqlalchemy`.

## 2 `GROUP BY` and Aggregations

SQL becomes much more powerful when you use it to aggregate and analyze data in various level of summarization (through the use of `GROUP BY`).

From our previous notebook, we shown the following syntax, where we have the `GROUP BY` and `HAVING` clauses:

```sql
SELECT <columns to return>
FROM <table>
WHERE <conditional filter statements>
GROUP BY <columns to group on>
HAVING <conditional filter statements that run after grouping>
ORDER BY <columns to sort on>
```

### Displaying Group Summaries

When computing aggregated statistics, there are two things that we want to specify - the desired level of aggregation (which columns values to group), and the aggregation value to compute. Examples of aggregate functions include the sum `SUM()`, average `AVG()`, count `COUNT()`, min `MIN()`, max `MAX()`, and many others. See [https://sqlite.org/lang_aggfunc.html](https://sqlite.org/lang_aggfunc.html) for the list of built-in aggregate functions for SQLite.

As an example, let's compute the number of rows in the `customer_purchases` table per market date per customer.

In [60]:
query = """
        SELECT
            market_date,
            customer_id,
            COUNT(*) AS items_purchased
        FROM customer_purchases
        GROUP BY market_date, customer_id
        ORDER BY market_date, customer_id
        LIMIT 10
        """
pd.read_sql(query, db)

,market_date,customer_id,items_purchased
0,2019-04-03,3,1
1,2019-04-03,4,1
2,2019-04-03,5,2
3,2019-04-03,6,2
4,2019-04-03,7,1
5,2019-04-03,9,5
6,2019-04-03,10,2
7,2019-04-03,11,1
8,2019-04-03,12,2
9,2019-04-03,16,2


`COUNT(*)` here just counts the number of rows in the group, which may not be an accurate measure of the total number of items purchased. If we mean by the total number of different products purchased by the customer at a given date at the market, we can change our query to:

In [61]:
query = """
        SELECT
            market_date,
            customer_id,
            COUNT(DISTINCT product_id) AS different_products_purchased
        FROM customer_purchases
        GROUP BY market_date, customer_id
        ORDER BY market_date, customer_id
        LIMIT 10
        """
pd.read_sql(query, db)

,market_date,customer_id,different_products_purchased
0,2019-04-03,3,1
1,2019-04-03,4,1
2,2019-04-03,5,2
3,2019-04-03,6,2
4,2019-04-03,7,1
5,2019-04-03,9,3
6,2019-04-03,10,2
7,2019-04-03,11,1
8,2019-04-03,12,2
9,2019-04-03,16,2


A pandas counterpart `groupby` method can also perform the same task.

In [65]:
query = """
        SELECT
            *
        FROM customer_purchases
        """
customer_purchases = pd.read_sql(query, db)

In [70]:
(customer_purchases.groupby(['market_date', 'customer_id']).product_id.nunique()
                   .reset_index().sort_values(by=['market_date', 'customer_id'])
                   .head(10))

,market_date,customer_id,product_id
0,2019-04-03,3,1
1,2019-04-03,4,1
2,2019-04-03,5,2
3,2019-04-03,6,2
4,2019-04-03,7,1
5,2019-04-03,9,3
6,2019-04-03,10,2
7,2019-04-03,11,1
8,2019-04-03,12,2
9,2019-04-03,16,2


### Performing Calculations Inside Aggregate Functions

We can also perform aggregations to created calculated columns prior to summarization.

<div class="alert alert-info">

**Example 1**
    
Calculate how much money the customer with id `3` spent on each `market_date` regardless of item or vendor.

</div>

In [71]:
query = """
        SELECT
            market_date,
            customer_id,
            SUM(quantity * cost_to_customer_per_qty) AS total_spent
        FROM customer_purchases
        WHERE customer_id = 3
        GROUP BY market_date
        ORDER BY market_date
        """
pd.read_sql(query, db)

,market_date,customer_id,total_spent
0,2019-04-03,3,4.0000
1,2019-04-13,3,56.0000
2,2019-04-24,3,20.0000
3,2019-04-27,3,72.0000
4,2019-05-01,3,52.0000
...,...,...,...
107,2020-09-23,3,36.0000
108,2020-09-26,3,13.7218
109,2020-09-30,3,18.0000
110,2020-10-03,3,22.0000


We can also perform `GROUP BY` on joined tables.

<div class="alert alert-info">

**Example 2**
    
Calculate the total amount each customer spent with each vendor, regardless of the items purchased.

</div>

In [74]:
query = """
        SELECT
            c.customer_id,
            c.customer_first_name,
            c.customer_last_name,
            v.vendor_name,
            v.vendor_id,
            ROUND(SUM(cp.quantity * cp.cost_to_customer_per_qty), 2) AS total_spent
        FROM customer AS c
            LEFT JOIN customer_purchases AS cp
                ON c.customer_id = cp.customer_id
            LEFT JOIN vendor AS v
                ON cp.vendor_id = v.vendor_id
        GROUP BY c.customer_id, v.vendor_id
        ORDER BY total_spent DESC, c.customer_id
        """
pd.read_sql(query, db)

,customer_id,customer_first_name,customer_last_name,vendor_name,vendor_id,total_spent
0,5,Abigail,Harris,Annie`s Pies,8,2728.0
1,3,Bob,Wilson,Annie`s Pies,8,2722.0
2,2,Manuel,Diaz,Annie`s Pies,8,2650.5
3,8,Norma,Valenzuela,Annie`s Pies,8,2646.0
4,24,Dawn,Nale,Annie`s Pies,8,2634.0
...,...,...,...,...,...,...
73,20,Valerie,Loftis,Fields of Corn,4,16.0
74,21,Duane,Sipp,Fields of Corn,4,13.5
75,23,Alvin,Laurie,Fields of Corn,4,13.5
76,24,Dawn,Nale,Fields of Corn,4,12.0


### Filtering with `HAVING`

After summarization, we can also filter the result according to the values after the aggregation functions where applied.

<div class="alert alert-info">

**Example 3**
    
List `market_dates` on which customer with id `3` spent at least $50.00, regarless of the items purchased or vendor.

</div>

In [4]:
query = """
        SELECT
            market_date,
            customer_id,
            SUM(quantity * cost_to_customer_per_qty) AS total_spent
        FROM customer_purchases
        WHERE customer_id = 3
        GROUP BY market_date
        HAVING total_spent >= 50
        ORDER BY market_date
        """
pd.read_sql(query, db)

,market_date,customer_id,total_spent
0,2019-04-13,3,56.0000
1,2019-04-27,3,72.0000
2,2019-05-01,3,52.0000
3,2019-05-11,3,108.0000
4,2019-05-22,3,96.5000
5,2019-07-03,3,62.1000
6,2019-07-06,3,89.4572
7,2019-08-03,3,57.0000
8,2019-09-21,3,86.5000
9,2019-10-02,3,60.5000


<img src="images/banner-down.png" style="width: 100%;">